In [11]:
import cvxpy as cp
import numpy as np

In [12]:
# define confidence level
CONFIDENCE_LEVEL = 0.95
RISK_ADVERSION_COEFFICIENT = 5
CVaR_MAX = 0.05

In [13]:
# Define problem data
# average returns of 3 assets
mu = np.array([0.1, 0.12, 0.15])
# covriance matrix of returns
Sigma = np.array([[0.005, 0.001, 0.002], [0.001, 0.006, 0.003], [0.002, 0.003, 0.009]])
# historical returns under different scenarios
R = np.array(
    [
        [0.08, 0.1, 0.12],
        [0.05, 0.07, 0.09],
        [0.12, 0.15, 0.18],
        [0.03, 0.02, 0.04],
        [0.15, 0.2, 0.25],
    ]
)

# Define variables
# weights
w = cp.Variable(3)
# Value at Risk
VaR = cp.Variable()
# excess loss beyond VaR under different scenarios
z = cp.pos(-R @ w - VaR)

# Define constraints
constraints = [
    cp.sum(w) == 1, # full investment
    w >= 0, # non-negativity of weights
    VaR + (1 / ((1-CONFIDENCE_LEVEL) * R.shape[0])) * cp.sum(z) <= CVaR_MAX, # Conditional Value at Risk constraint
]
# Define objective function
objective = cp.Maximize(mu @ w - RISK_ADVERSION_COEFFICIENT * cp.quad_form(w, Sigma))
# Solve problem
problem = cp.Problem(objective, constraints)
problem.solve()

np.float64(0.11007692307692309)

In [14]:
print("Optimal weights:", w.value)
print("Value at Risk (VaR):", VaR.value)
print("Conditional Value at Risk (CVaR):", VaR.value + (1 / ((1-CONFIDENCE_LEVEL) * R.shape[0])) * np.sum(z.value))

Optimal weights: [0.04615385 0.30769231 0.64615385]
Value at Risk (VaR): -0.0
Conditional Value at Risk (CVaR): 0.0
